First Attempt at Scraping Jomashop

In [9]:
# General Libraries
import os
import re
import time
import csv

# Analysis Libraries
import numpy as np
import pandas as pd

# Scraping Libraries
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent

Randomize Me to Prevent Getting Blocked

In [10]:
def random_soup(url):
    us = UserAgent()
    user_agent = {'User-Agent':us.random}
    response = requests.get(url,headers = user_agent)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    return soup

__Testing Jomashop__

In [3]:
watch_soup = random_soup('https://www.jomashop.com/seiko-watch-sur158p1.html')

Get Table of Watch Stats

In [4]:
watch_stats = watch_soup.find_all('span',class_='data')

In [5]:
watch_dict = dict()
for stat in watch_stats:
    watch_dict[stat['id']] = stat.text.replace('\n','').strip()

Get links to all pictures

In [6]:
image_links = watch_soup.find_all('a',href = re.compile('jpg'))

In [7]:
# f = open('00000001.jpg','wb')
# f.write(requests.get('http://www.gunnerkrigg.com//comics/00000001.jpg').content)
# f.close()

__ Testing Pretige Time__

In [17]:

#Get all Watch Links (Not Previously Saved)
#num_pages = 158
num_pages = 127
watch_list = []
for ix in range(1,num_pages+1):
    time.sleep(1+np.random.uniform(0,2)) 
    #listing_soup = random_soup('https://www.prestigetime.com/luxury-watches-for-men.html&page='+str(ix))
    listing_soup = random_soup('https://www.prestigetime.com/luxury-watches-for-women.html&page='+str(ix))
    listings = listing_soup.find_all('div',class_='thumbnail thumbnail-center')

    for listing in listings:

        watch_dict = dict()
        #URL
        watch_dict['url'] = listing.find('a')['href']

        #Image URL
        watch_dict['image_url'] = listing.find('img')['src']

        #Brand
        watch_dict['brand'] = listing.find('strong').text.strip()

        # Model Name 
        watch_dict['model_name'] = listing.find('span',id=re.compile("series-")).text.strip()

        # Model Number
        watch_dict['model_num'] = listing.find('span',id=re.compile("model_no")).text.strip()

        #Price
        price = listing.find('div',class_="caption-bottom").text
        price = price.split(':')[1]
        price = price.replace(',','')
        price = re.findall(r"(\d+)\.(\d+)", price)
        try:
            price = float(price[0][0] +'.'+ price[0][1])
        except:
            IndexError
            price = np.nan

        watch_dict['price'] = price 


        # Image Name
        image_name = watch_dict['brand'] + watch_dict['model_name'] + watch_dict['model_num']
        image_name = re.sub('[^0-9a-zA-Z]+', '', image_name)
        watch_dict['image_name'] = image_name

        watch_list.append(watch_dict)

        if image_name+'.jpg' not in os.listdir('prestige_time_pics/'):
            # Download Image
            us = UserAgent()
            user_agent = {'User-Agent':us.random}

            time.sleep(1+np.random.uniform(0,2)) 
            response = requests.get(watch_dict['image_url'],headers = user_agent)
            #f = open('prestige_time_pics/'+image_name+'.jpg','wb')
            f = open('prestige_time_pics_womens/'+image_name+'.jpg','wb')
            f.write(requests.get(watch_dict['image_url']).content)
            f.close()

    print(ix)

1
2
3
4
5
6
7
8
9


KeyboardInterrupt: 

In [ ]:
watch_DF = pd.DataFrame(watch_list)
watch_DF.to_csv('watch_page_list_womens.csv',index=False)

Test Individual Watch Listings

In [ ]:
watch_soup = random_soup(listing_links[501])

In [ ]:
stats = watch_soup.find('table',class_="table table-condensed item-table")

Table of Attributes

In [ ]:
stats_dict = dict()
for ix, row in enumerate(stats.find_all('tr')):
    tds = row.find_all('td')
    k = tds[0].text
    v = tds[1].text
    
    bad_vals = ['30m/99ft - 50m/165ft',
                '50m/165ft - 100m/330ft',
                '100m/330ft - 200m/660ft',
                '200m/660ft - 500m/1650ft',
                '500m/1650ft +',
               'No rating']
    if k == 'Movement':
        stats_dict[k] = v.split('\n')[0]
    elif k in bad_vals:
        break
    
    elif 'Water' in k:
        stats_dict['Water Resistance'] = row.find_all('td')[-1].text
    else:
        stats_dict[k] = v

In [ ]:
stats_dict